In [1]:
import ir_datasets
import requests
import aiohttp
import asyncio
import json
from aiolimiter import AsyncLimiter
from tqdm import tqdm

import asyncio
import random
import time

import aiohttp
from aiolimiter import AsyncLimiter

In [2]:
def get_id_from_url(url):
    if "list_uids=" in url:
        id = url.split("list_uids=")[1]
    elif "pubmed/" in url:
        id = url.split("pubmed/")[-1]
    elif "articles/" in url:
        splits = url.split("/")
        for split in splits:
            if split.startswith("PMC"):
                id = split[3:]
    elif "pubmedhealth/" in url:
        splits = url.split("/")
        for split in splits:
            if split.startswith("PMH"):
                id = split[3:]
    elif "from_uid=" in url:
        id = url.split("from_uid=")[1]
    else:
        print(url)
    return id


In [29]:
def write_line(line_dict):
    with open('metadata.jsonl', 'a+') as outfile:
        json.dump(line_dict, outfile)
        outfile.write('\n')


In [4]:
def request_metadata(pbid):
    base_url = f"https://api.semanticscholar.org/graph/v1/paper/PMID:{pbid}?fields=title,venue,authors,year,referenceCount,citationCount"
    result = requests.request("get", base_url.format(paper_id=pbid))
    if result.status_code == 200:
        return result.json()
    else:
        print(f"Error retrieving metadata for {pbid}. Status code: {result.status_code}")

In [5]:
urls = []
tripclick_ids = []

dataset = ir_datasets.load("tripclick/test")

for doc in dataset.docs_iter():
    urls.append(doc.url)
    tripclick_ids.append(doc.doc_id)

In [ ]:
pbids = []

for url in urls:
    pbid = get_id_from_url(url)
    pbids.append(pbid)

In [ ]:
done = []
with open('metadata.jsonl', 'r') as file:
    lines = file.readlines()

    for line in lines:
        try:
            done.append(json.loads(line)["tripclick_id"])
        except:
            print(line)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
len(done)

1513344

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"tripclick_id": tripclick_ids, "pbid": pbids})

In [ ]:
todo = df[~df["tripclick_id"].isin(done)]

In [ ]:
pbids = todo["pbid"].to_list()
tripclick_ids = todo["tripclick_id"].to_list()

In [ ]:
pbids_cleaned = []

for id in pbids:
    id = id.replace("&dopt=AbstractPlus", "")
    id = id.replace("?dopt=Abstract", "")
    id = id.replace("&dopt=Abstract", "")
    id = id.replace("&query_hl=2&itool=pubmed_docsum", "")

    try:
        int(id)
    except: 
        print(id)
    pbids_cleaned.append(id)

In [ ]:
len(pbids)

10534

In [ ]:
MAX_CONCURRENT = 50
RATE_LIMIT_IN_SECOND = 100
rate_limit = AsyncLimiter(RATE_LIMIT_IN_SECOND, 1)


errors = []
res = []

async def get(url, tripclick_id, session):
    try:
        response = await session.get(url=url)
        response.raise_for_status()


        metadata = await response.json()

        if metadata.get("message") == "Too Many Requests":
            print("Too Many Requests")

        # metadata["tripclick_id"] = tripclick_id
        # res.append(metadata)
        # write_line(metadata)

    except Exception as e:
        errors.append(tripclick_id)
        print("Unable to get url {} due to {}.".format(url, e.__class__))


async def worker(name, queue, session):
    while True:
        # Get a "work item" out of the queue.
        url, tripclick_id = await queue.get()

        # request and digest
        async with rate_limit:
            await get(url, tripclick_id, session)

        # Notify the queue that the "work item" has been processed.
        queue.task_done()


async def main_request_metadata(pbids, tripclick_ids):
    url = "https://api.semanticscholar.org/graph/v1/paper/PMID:{}?fields=title,venue,authors,year,referenceCount,citationCount"
    headers = {"x-api-key": "geheim"}
    connector = aiohttp.TCPConnector(limit=MAX_CONCURRENT)
    async with aiohttp.ClientSession(connector=connector, headers=headers) as session:
        # Create a queue that we will use to store our "workload".
        queue = asyncio.Queue()

        for pbid, tripclick_id in zip(pbids, tripclick_ids):
            queue.put_nowait((url.format(pbid), tripclick_id))


        # Create three worker tasks to process the queue concurrently.
        tasks = [
            asyncio.create_task(worker(f"worker-{idx}", queue, session))
            for idx in range(MAX_CONCURRENT)
        ]

        # Wait until the queue is fully processed.
        await queue.join()

        # Cancel our worker tasks.
        for task in tasks:
            task.cancel()

        # Wait until all worker tasks are cancelled.
        await asyncio.gather(*tasks, return_exceptions=True)

In [ ]:
await main_request_metadata(pbids_cleaned, tripclick_ids)

Unable to get url https://api.semanticscholar.org/graph/v1/paper/PMID:15084695?fields=title,venue,authors,year,referenceCount,citationCount due to <class 'aiohttp.client_exceptions.ClientResponseError'>.
Unable to get url https://api.semanticscholar.org/graph/v1/paper/PMID:16525140?fields=title,venue,authors,year,referenceCount,citationCount due to <class 'aiohttp.client_exceptions.ClientResponseError'>.
Unable to get url https://api.semanticscholar.org/graph/v1/paper/PMID:12672326?fields=title,venue,authors,year,referenceCount,citationCount due to <class 'aiohttp.client_exceptions.ClientResponseError'>.
Unable to get url https://api.semanticscholar.org/graph/v1/paper/PMID:15258070?fields=title,venue,authors,year,referenceCount,citationCount due to <class 'aiohttp.client_exceptions.ClientResponseError'>.
Unable to get url https://api.semanticscholar.org/graph/v1/paper/PMID:16860701?fields=title,venue,authors,year,referenceCount,citationCount due to <class 'aiohttp.client_exceptions.Cli

In [32]:
with open("errors.txt", "w") as file:
    for error in errors:
        file.write(error + "\n")
len(errors)

9360

In [4]:
errors = []
with open("errors.txt", "r") as file:
    a = file.read()

In [7]:
errors = a.split("\n")

In [24]:
error_docs = []

dataset = ir_datasets.load("tripclick/test")

with tqdm(total=dataset.docs_metadata()["count"]) as pbar:
    for doc in dataset.docs_iter():
        if doc.doc_id in errors:
            error_docs.append((doc, doc.doc_id))
        pbar.update(1)

100%|██████████| 1523878/1523878 [02:58<00:00, 8555.89it/s]


In [10]:
import requests

In [37]:
errors_pbm = []
def get(id, tripclick_id):
    url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id={id}&retmode=json"
    res = requests.get(url)
    if res.status_code == 200:
        try:
            res = res.json()
            res = res["result"][id]
            metadata = {
                "paperId": "", 
                "title": res.get("title"),
                "year": int(res.get("pubdate")[:4]),
                "referenceCount": "", 
                "citationCount": "", 
                "authors": res.get("authors"), 
                "tripclick_id": tripclick_id
                }
            return metadata
        except Exception as e:
            print(f"Error retrieving metadata for {id}. {e}")
            errors_pbm.append(tripclick_id)
    else:
        print(f"Error retrieving metadata for {id}. Status code: {res.status_code}")
        errors_pbm.append(tripclick_id)

In [34]:
error_docs.index((doc, doc_id))

73

In [38]:
with tqdm(total=len(error_docs[72:])) as pbar:
    for doc, doc_id in error_docs[72:]:
        metadata = get(doc_id, doc.doc_id)
        write_line(metadata)
        pbar.update(1)

  0%|          | 2/9288 [00:01<1:54:24,  1.35it/s]

Error retrieving metadata for 370262. 'NoneType' object is not subscriptable


  0%|          | 30/9288 [00:27<2:22:31,  1.08it/s]

Error retrieving metadata for 399260. 'NoneType' object is not subscriptable


  6%|▌         | 535/9288 [07:06<1:47:00,  1.36it/s]

...

100%|█████████▉| 9269/9288 [2:03:23<00:13,  1.37it/s]

Error retrieving metadata for 11601940. 'NoneType' object is not subscriptable


100%|█████████▉| 9273/9288 [2:03:28<00:19,  1.28s/it]

Error retrieving metadata for 11608988. 'NoneType' object is not subscriptable


100%|██████████| 9288/9288 [2:03:47<00:00,  1.25it/s]


In [42]:
with open('metadata_processed.jsonl', 'r') as file:
    lines = file.readlines()

In [43]:
len(lines)

1524989

# Unify Metadata

In [40]:
ids_processed = []
with tqdm(total=1_500_000) as pbar:
    with open('metadata.jsonl', 'r') as file:
        lines = file.readlines()
        for line in lines:
            try:
                ids_processed.append(json.loads(line)["tripclick_id"])
                with open('metadata_processed.jsonl', 'a+') as outfile:
                    json.dump(json.loads(line), outfile)
                    outfile.write('\n')
            except Exception as e:
                print(line)
                print(e)
            pbar.update(1)

 22%|██▏       | 330925/1500000 [00:53<03:40, 5311.34it/s]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

1516604it [04:23, 6237.44it/s]                             

null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable


1519500it [04:23, 7153.44it/s]

...

null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'Non

1523942it [04:24, 7160.17it/s]

null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable


1525080it [04:24, 5761.78it/s]

null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable
null

'NoneType' object is not subscriptable


# Split

In [15]:
years = []
with open('metadata.jsonl', 'r') as file:
    lines = file.readlines()

    for line in lines:
        try:
            metadata = json.loads(line)
            years.append(metadata["tripclick_id"], metadata["year"])
        except:
            print(line)
        break

{"paperId": "df269638af0c6febb7d26119043b80b6d9455d48", "title": "Volume progression in polycystic kidney disease.", "venue": "New England Journal of Medicine", "year": 2006, "referenceCount": 31, "citationCount": 286, "authors": [{"authorId": "2089459", "name": "J. Grantham"}, {"authorId": "144711413", "name": "V. Torres"}, {"authorId": "6042443", "name": "A. Chapman"}, {"authorId": "1398273666", "name": "L. Guay-Woodford"}, {"authorId": "2558976", "name": "K. Bae"}, {"authorId": "145115138", "name": "B. King"}, {"authorId": "143975679", "name": "L. Wetzel"}, {"authorId": "2032769", "name": "D. Baumgarten"}, {"authorId": "40582028", "name": "P. Kenney"}, {"authorId": "145418495", "name": "P. Harris"}, {"authorId": "152486634", "name": "S. Klahr"}, {"authorId": "115167475", "name": "W. Bennett"}, {"authorId": "32905822", "name": "G. Hirschman"}, {"authorId": "145086667", "name": "C. Meyers"}, {"authorId": "2118044456", "name": "Xiaoling Zhang"}, {"authorId": "2075368627", "name": "F. Z

### example

In [24]:
from aiolimiter import AsyncLimiter

MAX_CONCURRENT = 10
RATE_LIMIT_IN_SECOND = 1
rate_limit = AsyncLimiter(RATE_LIMIT_IN_SECOND, 10)


async def worker(name, queue):
    while True:
        # Get a "work item" out of the queue.
        sleep_for = await queue.get()

        async with rate_limit:
            await test()

        # Notify the queue that the "work item" has been processed.
        queue.task_done()


async def main():
    queue = asyncio.Queue()
    for _ in range(20):
        queue.put_nowait(1)

    # Create three worker tasks to process the queue concurrently.
    tasks = [
        asyncio.create_task(worker(f"worker-{idx}", queue))
        for idx in range(MAX_CONCURRENT)
    ]

    await queue.join()


    await asyncio.gather(*tasks, return_exceptions=True)


In [25]:
await main()

test
test
test
test
test
test
test


CancelledError: 

test
test
test
